In [1]:
from deps.dobot_api import DobotApiDashboard, DobotApiMove
from deps import utils
from deps import cv_core
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pynput import keyboard
import time
import ipyparallel as ipp

In [2]:
rc = ipp.Cluster(n=1).start_and_connect_sync()
e0 = rc[0]
e0.block = False
e0.activate('0')

Starting 1 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>
100%|██████████| 1/1 [00:05<00:00,  5.52s/engine]


In [46]:
dash = DobotApiDashboard('192.168.1.6', 29999)
move = DobotApiMove('192.168.1.6', 30003)
#keys = utils.Keyboard(dash)

In [47]:
dash.ClearError()
dash.EnableRobot()

'0,{},EnableRobot();'

In [21]:
keys = utils.Keyboard(dash)
keys.execute()

Position saved!
Position saved!
Position saved!
Position saved!
Special key pressed: Key.esc


In [22]:
well_plate = utils.assign_corners(keys.coords, reverse=True)
left_side_points = np.linspace(well_plate['ul'], well_plate['ll'], 12)[:,:2]
right_side_points = np.linspace(well_plate['ur'], well_plate['lr'], 12)[:,:2]
grid = []
for i in range(len(left_side_points)):
    x1, y1 = left_side_points[i]
    x2, y2 = right_side_points[i]
    a = (x2-x1)/(y2-y1)
    b = x1 - a*y1
    ys = np.linspace(y1,y2,8) 
    xs = a*ys + b
    grid += (list(zip(xs,ys)))

In [47]:
dash.ClearError()
dash.EnableRobot()
for coord in grid:
    x,y = coord
    move.MovL(x,y,-60,0)
    move.Sync()
    move.MovL(x,y,-80,0)
    move.Sync()
    move.MovL(x,y,-60,0)
    move.Sync()

In [132]:
xs = np.linspace(keys.coords[0][0], keys.coords[1][0], 8)
ys = np.linspace(keys.coords[0][1], keys.coords[1][1], 8)

for i, y in enumerate(ys):
    move.MovL(xs[i],y,-60,0)
    move.Sync()
    move.MovL(xs[i],y,-87,0)
    move.Sync()
    move.MovL(xs[i],y,-60,0)
    move.Sync()

In [30]:
dash.SpeedL(95)
for i in range(10):
    move.MovL(350, 0,0,0)
    move.MovL(250, 0, 0 ,0)
    move.Sync()

In [8]:

move.MovJ(220,100,50,0)

'0,{},MovJ(220.000000,100.000000,50.000000,0.000000);'

In [37]:
dash.GetAngle()

'0,{3.198121,31.791796,56.391380,-0.024719,0.000000,0.000000},GetAngle();'

In [129]:
dash.ClearError()
dash.EnableRobot()
#utils.correct_J4_angle(0, dash, move)
#move.RelJointMovJ(0,0,0,100,0,0)
#move.RelJointMovJ(0,0,0,60,0,0)
move.MovL(300, -20, -90,  0)

'0,{},Sync();0,{},MovL(278.608141,-139.504253,-95.000000,0.000000);'

In [21]:
utils.get_pose(dash)

X = 287.026297
Y = 9.547903
Z = -89.366798
r = 101.883568


array([287.026297,   9.547903, -89.366798, 101.883568])

In [32]:
dash.SpeedJ(95)
for i in range(10):
    move.RelMovJ(0,0,0,90)
    move.RelMovJ(0,0,0,-90)
    move.Sync()

In [56]:
dash.DisableRobot()

'0,{},DisableRobot();'

In [20]:
manmove = utils.ManualMove(move, dash)
manmove.execute()

In [7]:
manmove.coords

[array([251.906637, -20.562883, -90.098404,  -2.104246]),
 array([299.749476, -20.560973, -90.091187,  -2.104246]),
 array([307.674435,  39.513099, -90.083763,  -2.104259]),
 array([260.530755,  39.508798, -90.073135,  -2.104261])]

In [3]:
utils.calib_pos(move)

'0,{},MovL(250.000000,0.000000,-90.000000,0.000000);'

In [22]:
%%px0
from deps import utils
from deps.dobot_api import DobotApiDashboard, DobotApiMove
dash = DobotApiDashboard('192.168.1.6', 29999)
move = DobotApiMove('192.168.1.6', 30003)
manmove = utils.ManualMove(move, dash)
manmove.execute()
coords = manmove.coords

<AsyncResult(%px): pending>

In [23]:
def draw_cross(w,h,frame):
    w = int(w)
    h = int(h)
    frame[h-5:h+6, w-1:w+2] = [0, 255, 0]
    frame[h-1:h+2, w-5:w+6] = [0, 255, 0]
    return frame

cameraMatrix = np.load('./cam_matrices/cam_mtx.npy')
dist = np.load('./cam_matrices/dist.npy')
newCameraMatrix = np.load('./cam_matrices/newcam_mtx.npy')

cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, cv_core.camera_res_dict['1200'])
cv2.namedWindow('frame',  cv2.WINDOW_NORMAL)
cv2.resizeWindow('frame', 1348, 1011)

coords = []

while(True):
    ret, frame = cap.read()
    h,w = frame.shape[:-1]
    frame = cv2.undistort(frame, cameraMatrix, dist, None, newCameraMatrix)
    # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)s

    # dst = cv2.cornerHarris(gray, blockSize=2, ksize=3, k=0.04)
    # frame[dst > 0.01 * dst.max()] = [0, 255, 0]
    frame = draw_cross(w/2, h/2, frame)
    frame = draw_cross(w/4, h/4, frame)
    frame = draw_cross(3*w/4, h/4, frame)
    frame = draw_cross(w/4, 3*h/4, frame)
    frame = draw_cross(3*w/4, 3*h/4, frame)
    cv2.imshow('frame',frame)

    k = cv2.waitKey(1)
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[ WARN:0@418.942] global /opt/conda/conda-bld/opencv-suite_1656606573658/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


In [5]:
ar = e0.pull('coords')
coords = ar.get()
coords

[array([256.062558, -17.195119, -90.033806, -17.456869]),
 array([297.919911, -17.01596 , -90.018532, -17.456888]),
 array([304.430722,  37.666581, -90.009521, -17.456944]),
 array([263.810049,  37.101424, -89.996956, -17.457005])]

In [22]:
cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, cv_core.camera_res_dict['1200'])

cameraMatrix = np.load('./cam_matrices/cam_mtx.npy')
dist = np.load('./cam_matrices/dist.npy')
newCameraMatrix = np.load('./cam_matrices/newcam_mtx.npy')

cv2.namedWindow('frame',  cv2.WINDOW_NORMAL)
cv2.resizeWindow('frame', 1348, 1011)
template = cv2.imread('template.png', 0)

# anchors = [np.array([260,  40, -90,  0]),
#  np.array([250, -20, -90,  0]),
#  np.array([300, -20, -90,  0]),
#  np.array([307,  40, -90,  0])]

anchors = [np.array([307,  40, -90,  0]),
 np.array([260,  40, -90,  0]),
 np.array([250, -20, -90,  0]),
 np.array([300, -20, -90,  0])]

frame_num = 0
inter_idx = 0
recorded = []
x,y,z,r = anchors[0]
# move.MovL(x,y,z,r)
while(True):


    ret, frame = cap.read()
    frame = cv2.undistort(frame, cameraMatrix, dist, None, newCameraMatrix)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    plot_img = frame.copy()
    gray_copy = gray.copy()

    res = cv2.matchTemplate(gray_copy,template, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)
    cv2.rectangle(plot_img,top_left, bottom_right, 255, 2)

    # if frame_num % 15 == 0:
    #     x,y,z,r = anchors[inter_idx]
    #     move.MovL(x,y,z,r)
    #     move.Sync()
    #     inter_idx+=1
    #     if inter_idx == 4:
    #         inter_idx = 0
    #     if len(recorded) < 4:
    #         ret, frame = cap.read()
    #         frame = cv2.undistort(frame, cameraMatrix, dist, None, newCameraMatrix)
    #         plot_img = frame.copy()
    #         gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #         (minVal, maxVal, minLoc, maxLoc)=cv2.minMaxLoc(gray)
    #         recorded.append(maxLoc)

    # frame_num += 1

    # for anchor in recorded:
    #     cv2.circle(plot_img, anchor, 5, (255, 0, 0), 2)

    cv2.imshow('frame',plot_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()

# for anchor in recorded:
#     cv2.circle(frame, anchor, 5, (0, 255, 0), 2)
cv2.imshow('frame',frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

[ WARN:0@2017.813] global /opt/conda/conda-bld/opencv-suite_1656606573658/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


In [28]:
anchors = [np.array([307,  40, -90,  0]),
           np.array([260,  40, -90,  0]),
           np.array([250, -20, -90,  0]),
           np.array([300, -20, -90,  0])]

cameraMatrix = np.load('./cam_matrices/cam_mtx.npy')
dist = np.load('./cam_matrices/dist.npy')
newCameraMatrix = np.load('./cam_matrices/newcam_mtx.npy')
template = cv2.imread('template.png', 0)
w, h = template.shape[::-1]

# cap = cv2.VideoCapture(0)
# cap = cv_core.set_res(cap, cv_core.camera_res_dict['1200'])

# cv2.namedWindow('frame',  cv2.WINDOW_NORMAL)
# cv2.resizeWindow('frame', 1348, 1011)

dash.ClearError()
dash.EnableRobot()


recorded = []
for anchor in anchors:
    x,y,z,r = anchor
    move.MovL(x,y,z,r)
    move.Sync()

    #while(True):
    cap = cv2.VideoCapture(0)
    cap = cv_core.set_res(cap, cv_core.camera_res_dict['1200'])

    cv2.namedWindow('frame',  cv2.WINDOW_NORMAL)
    cv2.resizeWindow('frame', 1348, 1011)

    ret, frame = cap.read()
    frame = cv2.undistort(frame, cameraMatrix, dist, None, newCameraMatrix)
    plot_img = frame.copy()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    (minVal, maxVal, minLoc, maxLoc)=cv2.minMaxLoc(gray)

    res = cv2.matchTemplate(gray,template, cv2.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)

    mask = np.zeros_like(frame)
    cv2.rectangle(mask,top_left, bottom_right, (255,255,255), -1)
    result = cv2.bitwise_and(frame.astype('uint8'), mask.astype('uint8'))
    gray_result = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray_result,240,255,cv2.THRESH_BINARY)
    M = cv2.moments(thresh)
    if M["m00"] != 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        cv2.circle(plot_img, (cX, cY), 5, (0, 255, 0), 2)
        recorded.append((cX, cY))

    #cv2.circle(plot_img, maxLoc, 5, (0, 255, 0), 2)
    cv2.imshow('frame',plot_img)
    cv2.waitKey(0)
    cap.release()
    #cv2.destroyAllWindows()
    # if cv2.waitKey(1) & 0xFF == ord('q'):
    #     break
    
    #recorded.append(maxLoc)
#cap.release()
cv2.destroyAllWindows()

[ WARN:0@2223.131] global /opt/conda/conda-bld/opencv-suite_1656606573658/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ WARN:0@2226.004] global /opt/conda/conda-bld/opencv-suite_1656606573658/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ WARN:0@2228.271] global /opt/conda/conda-bld/opencv-suite_1656606573658/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
[ WARN:0@2230.467] global /opt/conda/conda-bld/opencv-suite_1656606573658/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


In [23]:
move.RelMovJ(0,  0, 0,  300)

'0,{},RelMovJ(0.000000,0.000000,0.000000,300.000000);'

In [24]:
utils.get_pose(dash)

X = 233.030666
Y = 34.727492
Z = 22.414635
r = 358.830139


array([233.030666,  34.727492,  22.414635, 358.830139])

In [29]:
# w = 1600
# h = 1200
xys = [(arr[0], arr[1]) for arr in anchors]
robot_coor = utils.assign_corners(xys, reverse=True)
#pix_coor = utils.assign_corners([(w/4, h/4), (3*w/4, h/4), (w/4, 3*h/4), (3*w/4, 3*h/4)])
pix_coor = utils.assign_corners(recorded)

features_mm_to_pixels_dict = {}
for key, value in robot_coor.items():
    features_mm_to_pixels_dict[value] = pix_coor[key]


tf_mtx = cv_core.compute_tf_mtx(features_mm_to_pixels_dict)

In [54]:
cont = cv_core.Contours()

def on_change(val): pass

offset = 40
cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, cv_core.camera_res_dict['1200'])

cameraMatrix = np.load('./cam_matrices/cam_mtx.npy')
dist = np.load('./cam_matrices/dist.npy')
newCameraMatrix = np.load('./cam_matrices/newcam_mtx.npy')

cv2.namedWindow('frame',  cv2.WINDOW_NORMAL)
cv2.resizeWindow('frame', 1348, 1011)
cv2.createTrackbar('Manual Lock', 'frame', 0, 1, on_change)
cv2.createTrackbar('Mask Offset', 'frame', offset, 150, on_change)
cv2.setMouseCallback('frame', cont.mousecallback)

idx = 0
prev_point = (0,0,0)
while(True):
    ret, frame = cap.read()
    frame = cv2.undistort(frame, cameraMatrix, dist, None, newCameraMatrix)
    val = cv2.getTrackbarPos('Manual Lock', 'frame')
    offset = cv2.getTrackbarPos('Mask Offset', 'frame')
    if val == 1:
        cont.locked = True
    else:
        cont.locked = False

    if cont.best_circ is None:
        while cont.best_circ is None:
            pt = cont.find_contours(frame, 10, offset)
    else:
        pt = cont.find_contours(frame, 10, offset)
        
    a,b,r = pt
    plot_img = frame.copy()
    cv2.circle(plot_img, (a, b), r, (3, 162, 255), 2)
    cv2.circle(plot_img, (a, b), 1, (0, 0, 255), 3)
    cv2.circle(plot_img, (a, b), r - offset, (0, 255, 0), 2)
    cv2.circle(plot_img, cont.big_circ[:2], cont.big_circ[2], (0, 0, 255), 2)
    cv2.putText(plot_img, f"{cont.big_circ[2]*2}px = 60mm", (cont.big_circ[0]-25, cont.big_circ[1] - cont.big_circ[2] - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
    
    cv2.drawContours(plot_img, cont.singular, -1,(0,255, 0),2)
    cv2.drawContours(plot_img, cont.clusters, -1,(0,0,255),2)

    if cont.selected:
        cv2.drawContours(plot_img, cont.selected, -1,(255,0,0),2)

    for c in cont.singular:
        # compute the center of the contour
        M = cv2.moments(c)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        # draw the contour and center of the shape on the image
        #cv2.drawContours(plot_img, [c], -1, (0, 255, 0), 2)
        cv2.circle(plot_img, (cX, cY), 2, (0, 0, 255), -1)
        # cv2.putText(plot_img, f"{cX},{cY}", (cX - 20, cY - 20),
        # cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        cv2.putText(plot_img, f"{cv2.contourArea(c)}", (cX - 20, cY - 20),
        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 1)
    #out.write(with_contours)

    for c in cont.clusters:
        # compute the center of the contour
        M = cv2.moments(c)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        # draw the contour and center of the shape on the image
        #cv2.drawContours(plot_img, [c], -1, (0, 255, 0), 2)
        cv2.circle(plot_img, (cX, cY), 2, (0, 0, 255), -1)
        # cv2.putText(plot_img, f"{cX},{cY}", (cX - 20, cY - 20),
        # cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)

        # cv2.putText(plot_img, f"{cv2.contourArea(c)}", (cX - 20, cY - 20),
        # cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 1)


    if prev_point is pt:
        idx += 1
    else:
        idx = 0
    prev_point = pt


    if idx >= 30:
        message = 'LOCKED'
        color = (0,255,0)
    else:
        message = 'SEARCHING'
        color = (0,0,255)

    cv2.putText(plot_img, "TARGET:", (25,25), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0), 2)
    cv2.putText(plot_img, f"{message}", (125,25), cv2.FONT_HERSHEY_SIMPLEX, 0.75, color, 2)
    cv2.putText(plot_img, f"Found: {len(cont.singular)}", (25,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0), 2)

    #cv2.setWindowProperty('frame', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    cv2.imshow('frame',plot_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[ WARN:0@3676.347] global /opt/conda/conda-bld/opencv-suite_1656606573658/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


In [59]:
dash.ClearError()
dash.EnableRobot()
X, Y, _ = tf_mtx @ (center[0], center[1], 1)
print(X,Y)
move.MovL(X, Y, -60, 0)
utils.correct_J4_angle(0, dash, move)

276.28277862707824 16.918359994713697


In [57]:
utils.get_pose(dash)

X = 276.282774
Y = 16.918357
Z = -59.999992
r = 3.504169


array([276.282774,  16.918357, -59.999992,   3.504169])

In [60]:
x,y,z,r = utils.get_pose(dash)
move.MovL(X,Y,-60,r+angles[0])
move.Sync()
utils.get_pose(dash)

X = 276.282774
Y = 16.918357
Z = -59.999992
r = 3.504169
X = 276.282774
Y = 16.91836
Z = -59.999992
r = 103.504166


array([276.282774,  16.91836 , -59.999992, 103.504166])

In [96]:
pos2 = utils.get_pose(dash)

X = 284.412914
Y = -81.050924
Z = -78.937172
r = -15.9008


In [51]:
dash.ClearError()
dash.EnableRobot()
move.MovL(300, -20, -90,  0)

'0,{},MovL(300.000000,-20.000000,-90.000000,0.000000);'

In [55]:
#centers_cuboids = cont.contour_centers(cont.singular)
angles = [200, 240, -250]
centers_cuboids = cont.contour_centers(cont.selected)

dash.ClearError()
dash.EnableRobot()

utils.default_pos(move)
move.Sync()
for idx, center in enumerate(centers_cuboids):
    X, Y, _ = tf_mtx @ (center[0], center[1], 1)
    move.MovL(X, Y, -60, 0)
    move.Sync()
    utils.correct_J4_angle(0, dash, move)
    move.Sync()
    x,y,z,r = utils.get_pose(dash)
    # move.MovL(X,Y,-60,r+angles[0])
    # move.Sync()
    move.MovL(X,Y,-89.5,r)
    move.Sync()
    move.MovL(X, Y, -89.5,r+160)
    move.Sync()
    move.MovL(X, Y, -60, r+160)
    move.Sync()
    x,y = grid[idx+8]
    move.MovL(x,y,-60, r+160)
    move.Sync()
    utils.correct_J4_angle(r+160, dash, move)
    move.Sync()
    x,y,z,r_new = utils.get_pose(dash)
    move.MovL(x, y, -85, r_new)
    move.Sync()
    utils.correct_J4_angle(0, dash, move)
    move.Sync()
    x,y,z,r_new = utils.get_pose(dash)
    # move.Sync()
    move.MovL(x, y, -60, r_new)
    move.Sync()

#utils.default_pos(move)

X = 281.199519
Y = 1.368729
Z = -60.000004
r = 0.278883
X = 224.578397
Y = -138.370443
Z = -59.999989
r = 128.640244
X = 224.578397
Y = -138.370443
Z = -84.999992
r = -31.638643
X = 277.695075
Y = 7.658427
Z = -59.999992
r = 1.579734
X = 225.502708
Y = -130.119314
Z = -60.0
r = 131.593964
X = 225.502681
Y = -130.119299
Z = -85.000008
r = -29.985756
X = 271.529993
Y = 7.895989
Z = -60.000019
r = 1.66567
X = 226.426991
Y = -121.868175
Z = -59.999992
r = 133.375519
X = 226.426991
Y = -121.868175
Z = -85.000015
r = -28.290152
X = 273.246595
Y = 1.605534
Z = -60.000008
r = 0.336653
X = 227.351267
Y = -113.617012
Z = -60.000004
r = 133.783417
X = 227.351267
Y = -113.617012
Z = -85.0
r = -26.55323
X = 280.790672
Y = 16.026213
Z = -60.000023
r = 3.26663
X = 228.275601
Y = -105.365906
Z = -59.999981
r = 138.489853
X = 228.275601
Y = -105.365906
Z = -84.999992
r = -24.776781
X = 292.809473
Y = 22.841933
Z = -60.0
r = 4.460584
X = 229.199909
Y = -97.114774
Z = -60.000011
r = 141.49762
X = 229.199

In [30]:
xs = np.linspace(keys.coords[0][0], keys.coords[1][0], 8)
ys = np.linspace(keys.coords[0][1], keys.coords[1][1], 8)

for i, y in enumerate(ys):
    move.MovL(xs[i],y,-60,0)
    move.Sync()
    move.MovL(xs[i],y,-87,0)
    move.Sync()
    move.MovL(xs[i],y,-60,0)
    move.Sync()

'0,{},MovL(290.412517,1.630035,-90.000000,0.000000);'

In [72]:
dash.ClearError()
dash.EnableRobot()
dash.AccL(20)
dash.SpeedL(20)
move.MovL(299.999996, -20.      -90, -89.999992,  0)

'0,{},MovL(299.999996,-110.000000,-89.999992,0.000000);'

In [34]:
utils.get_pose(dash)

X = 299.999996
Y = -20.0
Z = -89.999992
r = 0.0


array([299.999996, -20.      , -89.999992,   0.      ])

## Automatic laser detection

In [18]:
utils.get_pose(dash)

X = 282.801858
Y = 18.938397
Z = -60.000011
r = 0.0


array([282.801858,  18.938397, -60.000011,   0.      ])

In [ ]:
cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, cv_core.camera_res_dict['1200'])

cv2.namedWindow('frame',  cv2.WINDOW_NORMAL)
cv2.resizeWindow('frame', 1348, 1011)

while True:
    ret, frame = cap.read()
    cv2.imshow('frame',plot_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()